In [5]:
import pandas as pd
import numpy as np

In [185]:
# Get data
df = pd.DataFrame.from_csv("../data/Geokategoriserade händelser - data.csv")

# Drop duplicates
df = df.drop_duplicates()

# Drop calender aggregates
values_to_drop = ["Kungaparet", "Kronprinsessparet", "Prinsparet", "Prinsessparet"]
df = df.drop(df[df["person"].isin(values_to_drop)].index)


In [186]:
# Aggregate by person and country
by_person_country = pd.pivot_table(df, values=['date_start'], index=['person','country'], columns=['location_category'], aggfunc=np.count_nonzero)
by_person_country["totalt"] = by_person_country.sum(axis=1)

# Aggregate country
by_country = pd.pivot_table(df, values=['date_start'], index=['country'], columns=['location_category'], aggfunc=np.count_nonzero)
by_country["totalt"] = by_country.sum(axis=1)

# Only Swedes
swe_by_person = pd.pivot_table(df[df.country=="Sverige"], values=['date_start'], index=['person'], columns=['location_category'], aggfunc=np.count_nonzero)
swe_by_person["totalt"] = swe_by_person.sum(axis=1)


Det svenska kungahuset är det klart mest aktiva om man bara räknar aktiviterer (kalenderhändelser) per person.

In [187]:
print(by_country.sort_values("totalt", ascending=False))

                  date_start                 totalt
location_category    inrikes slottet utrikes       
country                                            
Sverige                  500     639      81   1220
Danmark                  373     235      58    666
Norge                    255     257      99    611


Och det svenska kungaparet är de mest aktiva.

In [188]:
df_top_active = by_person_country.sort_values('totalt', ascending=False)
print(df_top_active)

                                   date_start                 totalt
location_category                     inrikes slottet utrikes       
person                     country                                  
Kungen                     Sverige        127     222      18    367
Drottningen                Sverige        100     134      18    252
Kong Harald V              Norge           42     114      26    182
Kronprins Haakon           Norge           79      74      28    181
Prins Daniel               Sverige         86      78      15    179
Kronprinsessan             Sverige         67      92      20    179
H.M. Dronningen            Danmark         50      83      10    143
H.K.H. Kronprinsessen      Danmark         78      23      10    111
H.K.H. Kronprinsen         Danmark         62      34      11    107
Dronning Sonja             Norge           40      45      22    107
Prins Carl Philip          Sverige         58      40       6    104
Kronprinsesse Mette-Marit  Norge  

Men att bara jämföra antal kalenderhändelser kan vara lite missvisande eftersom kungahusen bokför lite olika aktivt, framför allt gällande mindre aktiviteter på slottet. Jämför vi kalenderhändelser utanför slottet nyanseras bilden något. 

Svenskarna är mest aktiva på inrikesevenemang.

In [189]:
print by_person_country.sort_values([('date_start', 'inrikes')], ascending=False).head(15)


                                  date_start                 totalt
location_category                    inrikes slottet utrikes       
person                    country                                  
Kungen                    Sverige        127     222      18    367
Drottningen               Sverige        100     134      18    252
Prins Daniel              Sverige         86      78      15    179
Kronprins Haakon          Norge           79      74      28    181
H.K.H. Kronprinsessen     Danmark         78      23      10    111
Kronprinsesse Mette-Marit Norge           73      15      15    103
Kronprinsessan            Sverige         67      92      20    179
H.K.H. Kronprinsen        Danmark         62      34      11    107
Prins Carl Philip         Sverige         58      40       6    104
H.K.H. Prins Joachim      Danmark         58      17       3     78
H.M. Dronningen           Danmark         50      83      10    143
Kong Harald V             Norge           42    

Medan norska kungahuset reser aktivast utomlands.

In [190]:
print by_person_country.sort_values([('date_start', 'utrikes')], ascending=False).head(15)


                                   date_start                 totalt
location_category                     inrikes slottet utrikes       
person                     country                                  
Kronprins Haakon           Norge           79      74      28    181
Kong Harald V              Norge           42     114      26    182
Dronning Sonja             Norge           40      45      22    107
Kronprinsessan             Sverige         67      92      20    179
Drottningen                Sverige        100     134      18    252
Kungen                     Sverige        127     222      18    367
Kronprinsesse Mette-Marit  Norge           73      15      15    103
Prins Daniel               Sverige         86      78      15    179
H.K.H. Kronprinsen         Danmark         62      34      11    107
H.K.H. Kronprinsessen      Danmark         78      23      10    111
H.M. Dronningen            Danmark         50      83      10    143
H.K.H. Prins Henrik        Danmark

Tittar vi enbart på svenska kungahuset är Sofia redan betydligt mer aktiv än Madeleine.

In [206]:
swe_by_person.sort_values("totalt", ascending=False)

date_start                 totalt
location_category       inrikes slottet utrikes       
person                                                
Kungen                      127     222      18    367
Drottningen                 100     134      18    252
Kronprinsessan               67      92      20    179
Prins Daniel                 86      78      15    179
Prins Carl Philip            58      40       6    104
Prinsessan Sofia             41      41     NaN     82
Prinsessan Madeleine         14      20       2     36
Christopher O'Neill           7      11       2     20
Prinsessan Estelle          NaN       1     NaN      1

Relativt sett (mätt i procent), är Kronprinsessan och Chris O'Neill är de mest utlandsaktiva svenskarna. 

In [213]:
swe_perc = swe_by_person.apply(lambda x: np.round(x / x.sum() * 2 * 100), axis=1)
swe_perc.sort_values([('date_start', 'utrikes')], ascending=False)

date_start                 totalt
location_category       inrikes slottet utrikes       
person                                                
Kronprinsessan               37      51      11    100
Christopher O'Neill          35      55      10    100
Prins Daniel                 48      44       8    100
Drottningen                  40      53       7    100
Prins Carl Philip            56      38       6    100
Prinsessan Madeleine         39      56       6    100
Kungen                       35      60       5    100
Prinsessan Estelle          NaN     100     NaN    100
Prinsessan Sofia             50      50     NaN    100